In [8]:
from datasets import load_from_disk
from transformers import (
    DataCollatorForSeq2Seq,
    RobertaTokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    T5ForConditionalGeneration,
)
import torch


In [9]:
torch.cuda.is_available()

False

In [2]:
tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-small")
model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-small")


In [5]:
dataset = load_from_disk("/data/nicolasmaier/dataset/hf_clean_seq_dataset")
dataset = dataset.remove_columns(["code", "contents", "xmi", "originalLine", "seq"])
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 385339
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 13524
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 22607
    })
})


In [6]:
BATCH_SIZE = 8

args = Seq2SeqTrainingArguments(
    output_dir="/data/nicolasmaier/model/codet5-finetuned-split",
    evaluation_strategy="steps",
    eval_steps=3000,
    logging_strategy="steps",
    logging_steps=3000,
    save_strategy="steps",
    save_steps=3000,
    learning_rate=5e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=0.01,
    save_total_limit=1000,
    num_train_epochs=3,
    predict_with_generate=True,
    # load_best_model_at_end=True,
    # metric_for_best_model="EM", # or BLEU?
    report_to="tensorboard",
    fp16=True,  # train faster
)

data_collator = DataCollatorForSeq2Seq(tokenizer)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["valid"].shuffle(seed=42).select(range(3000)),
    data_collator=data_collator,
    tokenizer=tokenizer,
)


/home/nicolasmaier/workspace/sse-transformers-mdd/venv/lib/python3.8/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
/home/nicolasmaier/workspace/sse-transformers-mdd/venv/lib/python3.8/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


ValueError: FP16 Mixed precision training with AMP or APEX (`--fp16`) and FP16 half precision evaluation (`--fp16_full_eval`) can only be used on CUDA devices.